# Introduction

This notebook shows an example of relaxing the LiFePO4 crystal.

If you are running this notebook from Google Colab, uncomment the next code box to install m3gnet first.

> Note: Because Google Colab's python version is sometimes rather old (v3.7 at the time of writing this notebook (Jul 2022), you will need to install m3gnet without dependencies and install the dependencies manually.

In [1]:
# !pip install --no-deps m3gnet
# !pip install pymatgen==2022.7.25 ase
# !pip install lxml

In [2]:
import warnings

import numpy as np
from pymatgen.ext.matproj import MPRester
from pymatgen.util.coord import pbc_diff

from m3gnet.models import Relaxer

for category in (UserWarning, DeprecationWarning):
    warnings.filterwarnings("ignore", category=category, module="tensorflow")

/Users/shyue/repos/pymatgen/pymatgen/analysis/phase_diagram.py:24: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
mpr = MPRester()
lfp = mpr.get_structure_by_material_id("mp-19017")  # This is LiFePO4.

lfp_strained = lfp.copy()  # We create a copy.
# Create a random strain between -5% and 5% for each direction
strains = np.random.uniform(low=-0.05, high=0.05, size=3)
lfp_strained.apply_strain(strains)
# In addition to the lattice strains, we also perturb the atoms by a distance of 0.1 angstrom.
lfp_strained.perturb(0.1)

In [4]:
relaxer = Relaxer()
relax_results: dict
%time relax_results = relaxer.relax(lfp_strained)
relaxed = relax_results["final_structure"]

2022-07-24 21:30:28.868092: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


CPU times: user 25.7 s, sys: 1min 4s, total: 1min 29s
Wall time: 13.2 s


Note that the relaxation only took < 20s.

In [5]:
print(f"Original lattice parameters are {[round(x, 3) for x in lfp.lattice.abc]}")
print(f"Strained lattice parameters are {[round(x, 3) for x in lfp_strained.lattice.abc]}")
print(f"Relaxed lattice parameters are {[round(x, 3) for x in relaxed.lattice.abc]}")

Original lattice parameters are [4.746, 10.444, 6.09]
Strained lattice parameters are [4.536, 10.363, 5.85]
Relaxed lattice parameters are [4.75, 10.47, 6.097]


In [6]:
print(f"Diff in fractional coords:\n{pbc_diff(lfp.frac_coords, relaxed.frac_coords)}")

Diff in fractional coords:
[[-4.64361707e-03 -3.06481782e-03  3.74647855e-03]
 [ 8.02859719e-03  6.26714898e-03  4.97933234e-03]
 [-1.31883698e-02 -8.83147324e-04  5.89795745e-03]
 [-4.64026395e-03  5.05661593e-03  4.40057152e-03]
 [-8.76794142e-03  2.45582773e-03  5.40328037e-05]
 [ 5.28374525e-04  1.68821361e-03  5.42106497e-03]
 [-2.45758950e-03  1.21124957e-03 -5.26549883e-03]
 [-1.42590821e-03  1.69523936e-03  3.48087402e-03]
 [-2.35524606e-04  1.18366297e-03  3.87536397e-03]
 [-3.66929741e-03  1.57373143e-03 -5.88734966e-03]
 [-2.48292289e-04  1.67496478e-03  3.69234433e-03]
 [-9.07207118e-04  9.57576002e-04  1.30374382e-03]
 [ 7.44593938e-04 -1.96943047e-03  3.87303571e-03]
 [-2.71164521e-03  4.00439554e-03 -3.19666586e-03]
 [-8.53971794e-04  5.80939056e-04 -1.79057068e-03]
 [-1.65764774e-03  1.07835726e-03  2.71340778e-03]
 [ 1.62494549e-03  1.70281445e-03  3.30065896e-03]
 [-1.58885350e-03  1.34992745e-03 -2.39858462e-03]
 [-5.26457219e-03  1.72202001e-03  6.43464849e-03]
 [-5

Quite clealy, the relaxation using the M3GNet universal IAP has brought the lattice parameters much closer to the original DFT one and the coordinates are also within $10^{-3}$ of the original fractional coordinates.